In [2]:
import torch
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [9]:
train_path = "a3_data/wsd_train.txt"
test_path = "a3_data/wsd_test_blind.txt"

with open(train_path, encoding = "utf-8") as f:
    for d, line in enumerate(f):
        print(line.lower())
        break
        
with open(test_path, encoding = "utf-8") as f:
    for d, line in enumerate(f):
        print(line.lower())
        break

keep%2:42:07::	keep.v	15	action by the committee in pursuance of its mandate , the committee will continue to keep under review the situation relating to the question of palestine and participate in relevant meetings of the general assembly and the security council . the committee will also continue to monitor the situation on the ground and draw the attention of the international community to urgent developments in the occupied palestinian territory , including east jerusalem , requiring international action .

?	physical.a	58	iaea pointed out that training and education were fundamental to the agency 's approach to enhancing physical protection systems in states . training courses , workshops and seminars that had been held on six continents had raised awareness and had provided hands-on experience of various subjects including the physical protection of research facilities , the practical operation of physical protection systems , and the engineering safety aspects of physical prote

# Representing the data

What is the best way to represent a piece of text for this task? CBoW, CNN, RNN, attention, ...?

Are stopwords able to change the sense of a word? I think so!

- standing in line - waiting for something
- standing in a line - they're just standing 

Based on this, I will not remove stopwords. I will also leave in punctuation, but it seems like a good idea to lowercase the entire text. We're not doing NER, and I don't want Line and line to end up having two meanings - the position alone should clarify the sense.

CBoW seems like a terrible choice in this setting - the word senses will almost certainly get lost. 
Since we know the position of the word, using a context window approach seems interesting. Try representation with GloVe
CNNs seem promising, as they can model interactions between words (exactly what we want). They also have a local structure, which is great. (can steal code from demo notebook if I want to use this)

Look at lecture before determining whether it's good to use RNN or attention models. 

In [ ]:
# Use the position of the target word, or not? can either include it in the embedding or just shove it all into a NN
# already need to use two different NN architectures, so maybe it's best to do a smart embedding straight away

In [61]:
# idea: only embed the sentence containing the word in question (a bit fiddly, maybe later)

# represent sentence/doc
# one-hot encode labels

# prediction: something with a softmax layer

In [81]:
sense_list = []
lemma_list = []
position_list = []
text_list = []

with open(train_path, encoding = "utf-8") as f:
    for d, line in enumerate(f):
        
        line = line.lower()
        
        ix = line.find("\t")
        sense_key = line[0:ix]
        line = line[ix+1:]
        
        ix = line.find("\t")
        lemma = line[0:ix]
        line = line[ix+1:]
        
        ix = line.find("\t")
        position = line[0:ix]
        text = line[ix+1:].split()
        
        if d == 0:
            print("sense_key ", sense_key)
            print("lemma ", lemma)
            print("position ", position)
            print("text ", text)
         
        sense_list.append(sense_key)
        lemma_list.append(lemma)
        position_list.append(position)
        text_list.append(text)
        
        #if d == 10000:
        #    break
        
print(d)

sense_key  keep%2:42:07::
lemma  keep.v
position  15
text  ['action', 'by', 'the', 'committee', 'in', 'pursuance', 'of', 'its', 'mandate', ',', 'the', 'committee', 'will', 'continue', 'to', 'keep', 'under', 'review', 'the', 'situation', 'relating', 'to', 'the', 'question', 'of', 'palestine', 'and', 'participate', 'in', 'relevant', 'meetings', 'of', 'the', 'general', 'assembly', 'and', 'the', 'security', 'council', '.', 'the', 'committee', 'will', 'also', 'continue', 'to', 'monitor', 'the', 'situation', 'on', 'the', 'ground', 'and', 'draw', 'the', 'attention', 'of', 'the', 'international', 'community', 'to', 'urgent', 'developments', 'in', 'the', 'occupied', 'palestinian', 'territory', ',', 'including', 'east', 'jerusalem', ',', 'requiring', 'international', 'action', '.']
76048


In [82]:
df = pd.DataFrame(sense_list, columns = ["Sense_key"])
df["Lemma"] = lemma_list
df["Position"] = position_list
df["Text"] = text_list

del sense_list, lemma_list, position_list

In [86]:
df.head()

,Sense_key,Lemma,Position,Text
0,keep%2:42:07::,keep.v,15,"[action, by, the, committee, in, pursuance, of..."
1,national%3:01:00::,national.a,25,"[a, guard, of, honour, stood, in, formation, i..."
2,build%2:31:03::,build.v,38,"[the, principle, that, statistics, should, be,..."
3,place%1:04:00::,place.n,36,"[again, ,, he, appealed, for, additional, supp..."
4,position%1:04:01::,position.n,76,"[also, ,, the, iaea, has, the, lowest, number,..."


In [54]:
df.Text[0]

['action',
 'by',
 'the',
 'committee',
 'in',
 'pursuance',
 'of',
 'its',
 'mandate',
 ',',
 'the',
 'committee',
 'will',
 'continue',
 'to',
 'keep',
 'under',
 'review',
 'the',
 'situation',
 'relating',
 'to',
 'the',
 'question',
 'of',
 'palestine',
 'and',
 'participate',
 'in',
 'relevant',
 'meetings',
 'of',
 'the',
 'general',
 'assembly',
 'and',
 'the',
 'security',
 'council',
 '.',
 'the',
 'committee',
 'will',
 'also',
 'continue',
 'to',
 'monitor',
 'the',
 'situation',
 'on',
 'the',
 'ground',
 'and',
 'draw',
 'the',
 'attention',
 'of',
 'the',
 'international',
 'community',
 'to',
 'urgent',
 'developments',
 'in',
 'the',
 'occupied',
 'palestinian',
 'territory',
 ',',
 'including',
 'east',
 'jerusalem',
 ',',
 'requiring',
 'international',
 'action',
 '.']

start out simple! ignore position, see it as a document classification problem

In [65]:
from collections import Counter


def count_word_frequencies(YOUR_FILE, ENCODING):
    
    freqs = Counter()
    with open(YOUR_FILE, encoding = ENCODING) as f:
        for line in f:
            tokens = line.lower().split()
            for token in tokens:
                freqs[token] += 1
                
    return freqs

In [66]:
def map_to_int(docs:list()) -> (list(), int, Counter()):
    
    '''
    Function from assignment 2.
    
    Create bag of words from cleaned and smaller corpus.
    Associate each word in bag with an unique integer,
    ranging from 0 (most common word) to length of bag of words.
    Map each token in docs to the respective int. Return this list of list of ints.
    '''
    
    freqs = Counter()
    for doc in docs:
        for token in doc:
            freqs[token] += 1
    most_common = freqs.most_common()
    
    token_to_int = []
    for i in range(len(most_common)):
        token_to_int.append(most_common[i][0])

    # Get pairs of elements    
    mapping = zip(token_to_int, range(0,len(token_to_int)))
    
    # Make pairs into a dictionary
    vocab = dict(mapping)
        
    # Match token to int
    docs_int = list()
    
    for doc in docs:
        docs_int.append(list([vocab.get(x) for x in doc]))
        
    return docs_int, len(vocab), vocab

In [67]:
texts_int, vocab_size, vocab = map_to_int(text_list)

In [73]:
texts_int[0][15]

220

In [72]:
vocab["keep"]

220

In [85]:
df["Text_int"] = texts_int

ValueError: Length of values does not match length of index

In [90]:
for sense in df.Sense_key.unique():
    print(len(df[df.Sense_key == sense]))

1155
205
425
33
436
442
429
442
4720
4103
529
490
448
281
436
429
420
218
448
278
435
437
425
425
425
292
50
170
437
38
427
680
28
556
433
481
426
32
440
429
456
440
528
198
2114
177
427
115
442
815
436
554
117
440
429
267
426
428
217
475
858
323
441
456
426
414
435
426
480
430
250
1052
436
428
426
703
437
427
177
433
458
32
454
437
469
448
433
228
192
442
425
410
96
431
456
445
425
448
430
91
80
429
591
210
430
473
425
426
451
470
121
442
379
457
255
148
429
418
436
428
439
428
426
242
326
431
442
425
425
199
441
279
129
440
428
430
191
382
180
240
430
24
431
431
44
179
252
470
80
442
112
426
196
448
259
80
430
91
48
120
430
472
430
136
439
55
89
208
130
51
48
426
40
191
124
25
63
45
147
30
427
481
34
108
122
51
55
24
87
68
430
129
131
69
26
69
23
53
92
38
43
33
92
31
58
22
55
57
51
34
63
37
44
22
29
36
46
29
30
41
22
22


Do I have to learn my own representation? Or can GloVe solve everything?
I don't control the context windows but maybe they've already solved this problem better than I can...

Could learn representation as I go, but there's not a lot of examples per unique sense_key, in some cases...
Also, this is a pain. Since the WSD texts appear to be generic enough, pretrained GloVe vectors should be ok. 
Do I use these as an initial guess or what? Also, GloVe only encodes one word at a time - so do I apply a context window myself as well?

for a 50-dim embedding of a 100-word document, we get a $100*50$ matrix. Seems to make sense to run a CNN over this! 
If the key word is on position 30, maybe have higher weights (gaussian?) for words close to it, so words 20-40 are weighted higher. 

output layer size should depend on the number of distinct senses for each lemma, so this is a lemma-by-lemma approach

or try summing up all the vectors to create one representation for the entire document, then input it into a 